In [1]:
#Dependencies and Setup
import pandas as pd
import math
import numpy as np
pd.set_option('display.max_columns', None)

#File to Load
train_data= "../02-Data_Files_Directory/02-Clean_Data/ptv_metro_train_station.csv"
listing_data= "../02-Data_Files_Directory/02-Clean_Data/airbnbFINAL_Swobabika.csv"

#files and store into Pandas data frame
station_df = pd.read_csv(train_data) 
station_df.head()

,STOP_ID,STOP_NAME,LATITUDE,LONGITUDE,TICKETZONE,ROUTEUSSP
0,19967,Anstey Railway Station (Brunswick),-37.761242,144.960684,1,Upfield
1,19968,Brunswick Railway Station (Brunswick),-37.767721,144.959587,1,Upfield
2,19969,Jewell Railway Station (Brunswick),-37.774987,144.958717,1,Upfield
3,19970,Royal Park Railway Station (Parkville),-37.781193,144.952301,1,Upfield
4,19971,Flemington Bridge Railway Station (North Melbo...,-37.788140,144.939323,1,Upfield


In [2]:
listing_df = pd.read_csv(listing_data, low_memory=False)
listing_df

,id,host_id,host_name,host_is_superhost,host_total_listings_count,street,neighbourhood_cleansed,city,state,zipcode,latitude,longitude,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,distance_from_cbd
0,12936,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,-37.859755,144.977369,Apartment,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",159.0,1253.0,4452.0,450.0,100.0,2,28.0,3,28,t,10,26,51,326,15,94,9,10,94,9,9,9,0.15,5.205174
1,47100,212071,Loren,t,4,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,-37.818037,145.004416,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",98.0,800.0,0.0,200.0,80.0,1,20.0,4,30,t,13,13,18,22,176,98,10,10,98,10,10,10,1.75,3.649571
2,50492,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,-37.861275,144.980379,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Buzzer/wireless int...",190.0,1743.0,5572.0,500.0,110.0,1,0.0,3,28,t,10,22,44,316,44,97,10,10,97,10,9,10,0.60,5.437003
3,51592,190879,Michelle,t,1,"Melbourne, VIC, Australia",Melbourne,Melbourne,Vic,3000,-37.814856,144.962672,Loft,4,1.5,1.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Paid parking off pr...",228.0,1575.0,5100.0,250.0,40.0,2,40.0,2,90,t,13,39,56,331,196,99,10,9,99,10,10,10,2.07,0.072335
4,66754,50121,Frank & Vince,t,17,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,-37.820916,144.993839,Apartment,4,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",138.0,1183.0,4172.0,450.0,100.0,2,28.0,3,28,t,11,19,49,324,43,95,9,10,95,10,10,9,0.46,2.794548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8922,30410904,127327853,Nathan,f,1,"Melbourne, VIC, Australia",Melbourne,Melbourne,Vic,3000,-37.808718,144.959530,Apartment,5,2.0,2.0,2.0,Real Bed,"{TV,Wifi,""Air conditioning"",Pool,Kitchen,Gym,E...",156.0,0.0,0.0,0.0,75.0,1,0.0,1,1125,t,15,32,52,52,2,100,10,10,100,10,10,10,2.00,0.700146
8923,30468406,30322920,Robyn,f,1,"Brunswick West, VIC, Australia",Moreland,Brunswick West,Vic,3055,-37.762380,144.949909,House,6,1.5,3.0,6.0,Real Bed,"{Wifi,Kitchen,""Free parking on premises"",Washe...",199.0,0.0,0.0,0.0,60.0,4,20.0,2,1125,t,15,34,46,97,1,0,0,0,0,0,0,0,1.00,5.893982
8924,30470482,36410227,Sabrina,f,276,"Port Melbourne, Australia",Port Phillip,Port Melbourne,Vic,3207,-37.844490,144.943306,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Pool,Kitc...",140.0,0.0,0.0,250.0,140.0,4,0.0,3,1125,t,2,25,55,146,1,0,0,0,0,0,0,0,1.00,3.780995
8925,30483362,154023219,Aishah,f,1,"Preston, VIC, Australia",Darebin,Preston,Vic,3072,-37.742043,145.002332,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,""Smoking a...",71.0,0.0,0.0,0.0,0.0,1,0.0,1,7,t,21,40,40,40,1,0,0,0,0,0,0,0,1.00,8.746854


In [3]:
station_df.dtypes

STOP_ID         int64
STOP_NAME      object
LATITUDE      float64
LONGITUDE     float64
TICKETZONE     object
ROUTEUSSP      object
dtype: object

In [4]:
listing_df.dtypes

id                               int64
host_id                          int64
host_name                       object
host_is_superhost               object
host_total_listings_count        int64
street                          object
neighbourhood_cleansed          object
city                            object
state                           object
zipcode                         object
latitude                       float64
longitude                      float64
property_type                   object
accommodates                     int64
bathrooms                      float64
bedrooms                       float64
beds                           float64
bed_type                        object
amenities                       object
price                          float64
weekly_price                   float64
monthly_price                  float64
security_deposit               float64
cleaning_fee                   float64
guests_included                  int64
extra_people             

In [5]:
from math import radians, cos, sin, asin, sqrt
def dist(lat1, long1, lat2, long2):

    # convert decimal degrees to radians 
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [6]:
def find_nearest(lat, long):
    distances = station_df.apply(
        lambda row: dist(lat, long, row['LATITUDE'], row['LONGITUDE']), 
        axis=1)
    return station_df.loc[distances.idxmin(), 'STOP_NAME']

# append the station name back to the listings file
listing_df['STOP_NAME'] = listing_df.apply(lambda row: find_nearest(row['latitude'], row['longitude']), axis=1)
# To check the data frame if it has a new column of station name (for each and every listing)
listing_df.head()

,id,host_id,host_name,host_is_superhost,host_total_listings_count,street,neighbourhood_cleansed,city,state,zipcode,latitude,longitude,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,distance_from_cbd,STOP_NAME
0,12936,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,-37.859755,144.977369,Apartment,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",159.0,1253.0,4452.0,450.0,100.0,2,28.0,3,28,t,10,26,51,326,15,94,9,10,94,9,9,9,0.15,5.205174,Windsor Railway Station (Windsor)
1,47100,212071,Loren,t,4,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,-37.818037,145.004416,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",98.0,800.0,0.0,200.0,80.0,1,20.0,4,30,t,13,13,18,22,176,98,10,10,98,10,10,10,1.75,3.649571,Burnley Railway Station (Burnley)
2,50492,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,-37.861275,144.980379,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Buzzer/wireless int...",190.0,1743.0,5572.0,500.0,110.0,1,0.0,3,28,t,10,22,44,316,44,97,10,10,97,10,9,10,0.60,5.437003,Windsor Railway Station (Windsor)
3,51592,190879,Michelle,t,1,"Melbourne, VIC, Australia",Melbourne,Melbourne,Vic,3000,-37.814856,144.962672,Loft,4,1.5,1.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Paid parking off pr...",228.0,1575.0,5100.0,250.0,40.0,2,40.0,2,90,t,13,39,56,331,196,99,10,9,99,10,10,10,2.07,0.072335,Flinders Street Railway Station (Melbourne City)
4,66754,50121,Frank & Vince,t,17,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,-37.820916,144.993839,Apartment,4,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",138.0,1183.0,4172.0,450.0,100.0,2,28.0,3,28,t,11,19,49,324,43,95,9,10,95,10,10,9,0.46,2.794548,Richmond Railway Station (Richmond)


In [7]:
listing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8927 entries, 0 to 8926
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           8927 non-null   int64  
 1   host_id                      8927 non-null   int64  
 2   host_name                    8927 non-null   object 
 3   host_is_superhost            8927 non-null   object 
 4   host_total_listings_count    8927 non-null   int64  
 5   street                       8927 non-null   object 
 6   neighbourhood_cleansed       8927 non-null   object 
 7   city                         8927 non-null   object 
 8   state                        8927 non-null   object 
 9   zipcode                      8927 non-null   object 
 10  latitude                     8927 non-null   float64
 11  longitude                    8927 non-null   float64
 12  property_type                8927 non-null   object 
 13  accommodates      

In [8]:
# create one dataframe with the new info
listing_final_df = pd.merge(listing_df,station_df[['STOP_NAME','LATITUDE','LONGITUDE']],on='STOP_NAME', how='left')
 
listing_final_df.head()

,id,host_id,host_name,host_is_superhost,host_total_listings_count,street,neighbourhood_cleansed,city,state,zipcode,latitude,longitude,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,distance_from_cbd,STOP_NAME,LATITUDE,LONGITUDE
0,12936,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,-37.859755,144.977369,Apartment,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",159.0,1253.0,4452.0,450.0,100.0,2,28.0,3,28,t,10,26,51,326,15,94,9,10,94,9,9,9,0.15,5.205174,Windsor Railway Station (Windsor),-37.856053,144.992035
1,47100,212071,Loren,t,4,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,-37.818037,145.004416,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",98.0,800.0,0.0,200.0,80.0,1,20.0,4,30,t,13,13,18,22,176,98,10,10,98,10,10,10,1.75,3.649571,Burnley Railway Station (Burnley),-37.827560,145.007555
2,50492,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,-37.861275,144.980379,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Buzzer/wireless int...",190.0,1743.0,5572.0,500.0,110.0,1,0.0,3,28,t,10,22,44,316,44,97,10,10,97,10,9,10,0.60,5.437003,Windsor Railway Station (Windsor),-37.856053,144.992035
3,51592,190879,Michelle,t,1,"Melbourne, VIC, Australia",Melbourne,Melbourne,Vic,3000,-37.814856,144.962672,Loft,4,1.5,1.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Paid parking off pr...",228.0,1575.0,5100.0,250.0,40.0,2,40.0,2,90,t,13,39,56,331,196,99,10,9,99,10,10,10,2.07,0.072335,Flinders Street Railway Station (Melbourne City),-37.818305,144.966964
4,66754,50121,Frank & Vince,t,17,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,-37.820916,144.993839,Apartment,4,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",138.0,1183.0,4172.0,450.0,100.0,2,28.0,3,28,t,11,19,49,324,43,95,9,10,95,10,10,9,0.46,2.794548,Richmond Railway Station (Richmond),-37.824074,144.990164


In [9]:
#    Calculate the great circle distance between two points 
#    on the earth (specified in decimal degrees)

from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [10]:
# Creating a new column to generate the output by passing lat long information to Haversine Equation
listing_final_df['station_distance'] = [haversine(listing_final_df.longitude[i],listing_final_df.latitude[i],listing_final_df.LONGITUDE[i],listing_final_df.LATITUDE[i]) for i in range(len(listing_final_df))]
listing_final_df['station_distance'] = listing_final_df['station_distance'].round(decimals=1)
# Printing the data table 
listing_final_df.head()


,id,host_id,host_name,host_is_superhost,host_total_listings_count,street,neighbourhood_cleansed,city,state,zipcode,latitude,longitude,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,distance_from_cbd,STOP_NAME,LATITUDE,LONGITUDE,station_distance
0,12936,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,-37.859755,144.977369,Apartment,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",159.0,1253.0,4452.0,450.0,100.0,2,28.0,3,28,t,10,26,51,326,15,94,9,10,94,9,9,9,0.15,5.205174,Windsor Railway Station (Windsor),-37.856053,144.992035,1.4
1,47100,212071,Loren,t,4,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,-37.818037,145.004416,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",98.0,800.0,0.0,200.0,80.0,1,20.0,4,30,t,13,13,18,22,176,98,10,10,98,10,10,10,1.75,3.649571,Burnley Railway Station (Burnley),-37.827560,145.007555,1.1
2,50492,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,-37.861275,144.980379,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Buzzer/wireless int...",190.0,1743.0,5572.0,500.0,110.0,1,0.0,3,28,t,10,22,44,316,44,97,10,10,97,10,9,10,0.60,5.437003,Windsor Railway Station (Windsor),-37.856053,144.992035,1.2
3,51592,190879,Michelle,t,1,"Melbourne, VIC, Australia",Melbourne,Melbourne,Vic,3000,-37.814856,144.962672,Loft,4,1.5,1.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Paid parking off pr...",228.0,1575.0,5100.0,250.0,40.0,2,40.0,2,90,t,13,39,56,331,196,99,10,9,99,10,10,10,2.07,0.072335,Flinders Street Railway Station (Melbourne City),-37.818305,144.966964,0.5
4,66754,50121,Frank & Vince,t,17,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,-37.820916,144.993839,Apartment,4,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",138.0,1183.0,4172.0,450.0,100.0,2,28.0,3,28,t,11,19,49,324,43,95,9,10,95,10,10,9,0.46,2.794548,Richmond Railway Station (Richmond),-37.824074,144.990164,0.5


In [11]:
# ***DEPRICATED*** merge the station distance and final data instead
#listing_final_df[['id', 'latitude', 'longitude', 'STOP_NAME', 'LATITUDE', 'LONGITUDE', 'station_distance']].to_csv('../02-Data_Files_Directory/02-Clean_Data/listing_station_coords.csv')
listing_final_df

,id,host_id,host_name,host_is_superhost,host_total_listings_count,street,neighbourhood_cleansed,city,state,zipcode,latitude,longitude,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,distance_from_cbd,STOP_NAME,LATITUDE,LONGITUDE,station_distance
0,12936,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,-37.859755,144.977369,Apartment,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",159.0,1253.0,4452.0,450.0,100.0,2,28.0,3,28,t,10,26,51,326,15,94,9,10,94,9,9,9,0.15,5.205174,Windsor Railway Station (Windsor),-37.856053,144.992035,1.4
1,47100,212071,Loren,t,4,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,-37.818037,145.004416,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",98.0,800.0,0.0,200.0,80.0,1,20.0,4,30,t,13,13,18,22,176,98,10,10,98,10,10,10,1.75,3.649571,Burnley Railway Station (Burnley),-37.827560,145.007555,1.1
2,50492,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,-37.861275,144.980379,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Buzzer/wireless int...",190.0,1743.0,5572.0,500.0,110.0,1,0.0,3,28,t,10,22,44,316,44,97,10,10,97,10,9,10,0.60,5.437003,Windsor Railway Station (Windsor),-37.856053,144.992035,1.2
3,51592,190879,Michelle,t,1,"Melbourne, VIC, Australia",Melbourne,Melbourne,Vic,3000,-37.814856,144.962672,Loft,4,1.5,1.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Paid parking off pr...",228.0,1575.0,5100.0,250.0,40.0,2,40.0,2,90,t,13,39,56,331,196,99,10,9,99,10,10,10,2.07,0.072335,Flinders Street Railway Station (Melbourne City),-37.818305,144.966964,0.5
4,66754,50121,Frank & Vince,t,17,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,-37.820916,144.993839,Apartment,4,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",138.0,1183.0,4172.0,450.0,100.0,2,28.0,3,28,t,11,19,49,324,43,95,9,10,95,10,10,9,0.46,2.794548,Richmond Railway Station (Richmond),-37.824074,144.990164,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8922,30410904,127327853,Nathan,f,1,"Melbourne, VIC, Australia",Melbourne,Melbourne,Vic,3000,-37.808718,144.959530,Apartment,5,2.0,2.0,2.0,Real Bed,"{TV,Wifi,""Air conditioning"",Pool,Kitchen,Gym,E...",156.0,0.0,0.0,0.0,75.0,1,0.0,1,1125,t,15,32,52,52,2,100,10,10,100,10,10,10,2.00,0.700146,Melbourne Central Railway Station (Melbourne C...,-37.809939,144.962594,0.3
8923,30468406,30322920,Robyn,f,1,"Brunswick West, VIC, Australia",Moreland,Brunswick West,Vic,3055,-37.762380,144.949909,House,6,1.5,3.0,6.0,Real Bed,"{Wifi,Kitchen,""Free parking on premises"",Washe...",199.0,0.0,0.0,0.0,60.0,4,20.0,2,1125,t,15,34,46,97,1,0,0,0,0,0,0,0,1.00,5.893982,Anstey Railway Station (Brunswick),-37.761242,144.960684,1.0
8924,30470482,36410227,Sabrina,f,276,"Port Melbourne, Australia",Port Phillip,Port Melbourne,Vic,3207,-37.844490,144.943306,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Pool,Kitc...",140.0,0.0,0.0,250.0,140.0,4,0.0,3,1125,t,2,25,55,146,1,0,0,0,0,0,0,0,1.00,3.780995,Southern Cross Railway Station (Melbourne City),-37.817936,144.951411,3.0
8925,30483362,154023219,Aishah,f,1,"Preston, VIC, Australia",Darebin,Preston,Vic,3072,-37.742043,145.002332,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,""Smoking a...",71.0,0.0,0.0,0.0,0.0,1,0.0,1,7,t,21,40,40,40,1,0,0,0,0,0,0,0,1.00,8.746854,Preston Railway Station (Presto

In [12]:
# # export main data with station data to airbnbFINAL_Swobabika.csv
listing_final_df.to_csv("../02-Data_Files_Directory/02-Clean_Data/airbnbFINAL_Swobabika.csv", index =False)